Required libraries:

In [1]:
import pandas as pd
import numpy
from DecisionTree import *
import copy
import matplotlib.pyplot as plt
import time

<h1> Decision Tree Project </h1>
In this project we are trying to train a model based on Decision Trees for predicting satistaction in public flights.

we split the project in three main parts: 

1- Data Cleaning and Oraganization

2- Building the Tree

3- Evaluation and Benchmarking

<hr>

<h3>Data Handeling </h3>

There are several operations needed to be preformed on data to make it ready for training.

First we should categorize continuous variables and limit their domains. there are many ways to do this but in this project we split continuous varibles such that each category contains a fixed precentage of the total data. this way there is a lower bound on the given data for every decision we make in the tree. this precentage is a hyper parameter of the tree and can be changed.

In [2]:
def categorize(series: pd.Series, precentInGroup):
    # categorzies a single column of dataframe with a given percentage
    CountInGroup = int(len(series) * precentInGroup)
    count = int(1 / precentInGroup)
    series = series.sort_values()
    bins = [-math.inf]
    border = 0
    for i in range(int(count)):
        bins.append(series.iloc[border])
        border += CountInGroup
    bins.append(math.inf)
    series = pd.cut(series, bins, duplicates="drop")
    out = series.astype(str).sort_index()
    return (out, bins)

def Categorization1(TrainData, TestData, features, categorizedFeatures):
    # this function is the categorization procedure. calculate the category bounds for
    # every continuous feature in train data. then it applies those bounds for test data.
    # returns feature domains for every feature in data. 
    featuresDomains = {}
    for feature in features:
        if feature == "id" or feature == "Unnamed: 0":
            continue
        if feature in categorizedFeatures:
            TrainData[feature], bins = categorize(TrainData[feature], 0.3)
            TestData[feature] = series = pd.cut(TestData[feature], bins, duplicates="drop").astype(str)
        series = TrainData[feature]
        domain = set()
        for item in series:
            if item not in domain:
                domain.add(item)
        featuresDomains[feature] = domain
    return featuresDomains

After categorizing we should encode variable domains to integers. this is because string comparison is much slower than integer comparison.

In [3]:
def EncodeData(data: pd.DataFrame, featureDomains):
    # Encodes every feature domain in data to range(0...n) 
    for feature in featureDomains:
        domain = featureDomains[feature]
        featureDomains[feature] = list(range(len(domain)))
        ReplaceList = {}
        for encoding, item in enumerate(domain):
            ReplaceList[item] = encoding
        data[feature] = data[feature].replace(ReplaceList)

At last we combine these functions to clean our train and test data.

In [4]:
def CleanDataV1(TrainData, TestData, features, categorizedFeatures):
    featureDomains = Categorization1(TrainData, TestData, features, categorizedFeatures)
    EncodeData(TestData, copy.deepcopy(featureDomains))
    EncodeData(TrainData, featureDomains)
    return featureDomains

<hr>

<h3>Making The Tree</h3>
our tree structure consist of two main components:

1- Decision Nodes

2- Leaf Nodes

in Decision Nodes we make a decision for a fixed feature and decend the three to lower depths.

in Leaf Nodes we classify a entry based on the probability distribution that lives in the node.

because there is the chance of overfitting in decision trees we would like to somehow prune the made tree. in this project we preprune the tree by limiting the depth of tree. this makes depth our second hyperparameter. we build the tree by recursively finding the most important feature 
spliting the database based on its value. when our depth exceeds a certain threshold or the gini index / entropy of the feature equals zero we should make a leafnode based on the distribution of labels in the parent node.



In [ ]:
# tree making procedure (do not run this)
self.feature = self.pickBestFeature(importance)
self.used.add(self.feature)
self.makeChilds()
self.used.remove(self.feature)

to pick the best feature we iterate though each not fixed feature and pick the one with the most gain in importance.

In [ ]:
def pickBestFeature(self, importance):
        bestFeature = None
        MaximumGain = -math.inf 
        for feature in self.domains:
            if feature not in self.used:
                gain = self.calculateGain(feature, importance)
                if MaximumGain < gain:
                    MaximumGain = gain
                    bestFeature = feature
        return bestFeature

to calculate gain for each feature, first we average the importance of a nodes children
then we calculate the diffrence between this weighted average and the nodes importance.

In [ ]:
def calculateGain(self, feature, importance):
        totalChildImportance = 0
        domain = self.domains[feature]
        for item in domain:
            nextData = query(self.data, feature, item)
            if len(nextData) > 0:
                totalChildImportance += len(nextData) * importance(self.labels, nextData)
        averageChildImportance = totalChildImportance / len(domain)
        parentImportance = importance(self.labels, self.data)
        return parentImportance - averageChildImportance

we use gini index or information gain to calculate importance. this is due to the fact that 
a node with lower importance means that the data is more uniformly distributed.

In [ ]:
def calculateGini(labels, data: pd.DataFrame):
        Gini = 0
        for label in labels:
            prob = len(query(data, "satisfaction", label)) / len(data)
            if prob == 0:
                continue
            Gini += prob * (1 - prob)
        return Gini

def calculateEntropy(labels, data: pd.DataFrame):
        entorpy = 0
        for label in labels:
            prob = len(query(data, "satisfaction", label)) / len(data)
            if prob == 0:
                continue
            entorpy += prob * math.log2(prob)
        return -entorpy

after finding the best feature, we split the from it. and recurviely make the tree until the depth threshold.

In [ ]:
def makeChilds(self):
        for item in self.domains[self.feature]:
            nextData = query(self.data, self.feature, item)
            if len(nextData) != 0:
                entropy = self.importance(self.labels, nextData)
            if len(nextData) == 0:
                self.Childs[item] = LeafNode(self.labels, self.data)
            elif entropy == 0 or self.depth == 0:
                self.Childs[item] = LeafNode(self.labels, nextData)
            else:
                self.Childs[item] = DecisionNode(nextData, self.domains, self.labels, self.depth - 1, self.used, self.importance)


<hr>

<h3>Evaluation & Benchmarking </h3>

after successfuly building the tree, its time for measuring its performance. first we write a function for classifying a single data point.

In [ ]:
def classify(self, entry):
        value = entry[self.feature]
        return self.Childs[value].classify(entry)

now we make two procedures for training and testing with diffrenct hyper parameters.

In [7]:
def Train(TrainData, featuresDomains, depth, importance):
    return DesicionTree(featuresDomains, TrainData, featuresDomains["satisfaction"], depth, set(["satisfaction"]), importance)

def Test(TestData, tree):
    #returns the accuracy of the model
    correct = 0
    for i in range(len(TestData)):
        entry = TestData.iloc[i]
        correct += tree.classify(entry) == TestData.iloc[i]["satisfaction"]
    return (correct / len(TestData)) * 100

we can finally import our data and clean it.

In [5]:
DataPath = "Data/Airplane.csv"
DataFrame = pd.read_csv(DataPath)
TestData = DataFrame.iloc[:2000]
TrainData = DataFrame.iloc[2000:]
features = TestData.keys()
categorizedFeatures = set(["Age",
                           "Flight Distance",
                           "Departure Delay in Minutes",
                           "Arrival Delay in Minutes"])
featuresDomains = CleanDataV1(TrainData, TestData, features, categorizedFeatures)

c:\Users\user\anaconda3\envs\cs188\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\Users\user\anaconda3\envs\cs188\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\Users\user\anaconda3\envs\cs188\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

now we can finally measure our accuracy.

In [9]:
depth = 7
tree = Train(TrainData, featuresDomains, depth, DecisionNode.calculateEntropy)
testAccuracy = Test(TestData, tree)
trainAccuracy = Test(TrainData, tree)
print(f"Test Accuracy: {testAccuracy}\nTrain Accuracy: {trainAccuracy}")

Test Accuracy: 10.0
Train Accuracy: 91.16913958235202
